# Setup (Imports)

In [ ]:
!pip install python-dotenv
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json

import os
from datetime import datetime, timedelta, timezone
import asyncio

from google.colab import userdata
pwd = userdata.get('open_ai_calati')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from openai import OpenAI
from neo4j import GraphDatabase

# Initialize Neo4j driver
uri = "neo4j+s://c87e3ce1.databases.neo4j.io"
username = "neo4j"
password = "zW13mCVhTWkcQLhIDoiKloD07IUg0W5TiszGzKwjCXA"
driver = GraphDatabase.driver(uri, auth=(username, password))

# Initialize OpenAI client
#client = OpenAI(api_key="sk-proj-wVJB8gpPHgjqaCOnZYPxT3BlbkFJmeQUAK5CNJNpWvdu1wWf")

In [ ]:
# !pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


# Data Loading

In [ ]:
train_df_path = '/content/drive/MyDrive/DATASCI 266/266 Final Project/redial_dataset/train_data.jsonl'

# Load the JSONL data
data_train = []
with open(train_df_path, 'r') as f:
    for line in f:
        data_train.append(json.loads(line.strip()))

# Check the first item to confirm loading
print(data_train[0])

{'movieMentions': {'203371': 'Final Fantasy: The Spirits Within (2001)', '84779': 'The Triplets of Belleville (2003)', '122159': 'Mary and Max (2009)', '151313': 'A Scanner Darkly  (2006)', '191602': 'Waking Life (2001)', '165710': 'The Boss Baby (2017)'}, 'respondentQuestions': {'203371': {'suggested': 1, 'seen': 0, 'liked': 1}, '84779': {'suggested': 0, 'seen': 1, 'liked': 1}, '122159': {'suggested': 0, 'seen': 1, 'liked': 1}, '151313': {'suggested': 0, 'seen': 1, 'liked': 1}, '191602': {'suggested': 0, 'seen': 1, 'liked': 1}, '165710': {'suggested': 1, 'seen': 0, 'liked': 1}}, 'messages': [{'timeOffset': 0, 'text': "Hi there, how are you? I'm looking for movie recommendations", 'senderWorkerId': 0, 'messageId': 1021}, {'timeOffset': 15, 'text': 'I am doing okay. What kind of movies do you like?', 'senderWorkerId': 1, 'messageId': 1022}, {'timeOffset': 66, 'text': 'I like animations like @84779 and @191602', 'senderWorkerId': 0, 'messageId': 1023}, {'timeOffset': 86, 'text': 'I also 

In [ ]:
test_df_path = '/content/drive/MyDrive/DATASCI 266/266 Final Project/redial_dataset/test_data.jsonl'

# Load the JSONL data
data_test = []
with open(test_df_path, 'r') as f:
    for line in f:
        data_test.append(json.loads(line.strip()))

# Check the first item to confirm loading
print(data_test[0])

{'movieMentions': {'111776': 'Super Troopers (2001)', '91481': 'Beverly Hills Cop (1984)', '151656': 'Police Academy  (1984)', '134643': 'American Pie  (1999)', '192131': 'American Pie ', '124771': '48 Hrs. (1982)', '94688': 'Police Academy 2: Their First Assignment (1985)', '101794': 'Lethal Weapon (1987)'}, 'respondentQuestions': {'111776': {'suggested': 0, 'seen': 1, 'liked': 1}, '91481': {'suggested': 1, 'seen': 2, 'liked': 2}, '151656': {'suggested': 1, 'seen': 0, 'liked': 1}, '134643': {'suggested': 0, 'seen': 1, 'liked': 1}, '192131': {'suggested': 0, 'seen': 1, 'liked': 1}, '124771': {'suggested': 1, 'seen': 2, 'liked': 2}, '94688': {'suggested': 1, 'seen': 0, 'liked': 1}, '101794': {'suggested': 1, 'seen': 0, 'liked': 2}}, 'messages': [{'timeOffset': 0, 'text': 'Hi I am looking for a movie like @111776', 'senderWorkerId': 956, 'messageId': 204171}, {'timeOffset': 48, 'text': 'You should watch @151656', 'senderWorkerId': 957, 'messageId': 204172}, {'timeOffset': 90, 'text': 'Is

In [ ]:
# Function to print a conversation
def display_conversation(conversation):
    print(f"Conversation ID: {conversation['conversationId']}")
    print("Initiator:", conversation['initiatorWorkerId'])
    print("Respondent:", conversation['respondentWorkerId'])
    print("\nMessages:\n")

    for message in conversation['messages']:
        sender = "Initiator" if message['senderWorkerId'] == conversation['initiatorWorkerId'] else "Respondent"
        print(f"{sender} (Worker ID: {message['senderWorkerId']}):")
        print(f"  Message ID: {message['messageId']}")
        print(f"  Time Offset: {message['timeOffset']}s")
        print(f"  Text: {message['text']}\n")

# Display the first conversation
display_conversation(data_test[0])

Conversation ID: 20001
Initiator: 956
Respondent: 957

Messages:

Initiator (Worker ID: 956):
  Message ID: 204171
  Time Offset: 0s
  Text: Hi I am looking for a movie like @111776

Respondent (Worker ID: 957):
  Message ID: 204172
  Time Offset: 48s
  Text: You should watch @151656

Initiator (Worker ID: 956):
  Message ID: 204173
  Time Offset: 90s
  Text: Is that a great one? I have never seen it. I have seen @192131

Initiator (Worker ID: 956):
  Message ID: 204174
  Time Offset: 122s
  Text: I mean @134643

Respondent (Worker ID: 957):
  Message ID: 204175
  Time Offset: 180s
  Text: Yes @151656 is very funny and so is @94688

Initiator (Worker ID: 956):
  Message ID: 204176
  Time Offset: 199s
  Text: It sounds like I need to check them out

Respondent (Worker ID: 957):
  Message ID: 204177
  Time Offset: 219s
  Text: yes you will enjoy them

Initiator (Worker ID: 956):
  Message ID: 204178
  Time Offset: 253s
  Text: I appreciate your time. I will need to check those out. Are t

Currently, data entries have multiple recommendations. We have split the recommendations to cutoff at the point of each recommendation.

We will also add mentions from the initiator

In [ ]:
from copy import deepcopy
import re

# Function to split conversation and update all relevant fields without overlap, including initiator movie mentions
def split_conversations_no_overlap_with_initiator_mentions(data):
    new_data = []

    for conversation in data:
        messages = conversation['messages']
        respondent_id = conversation['respondentWorkerId']
        initiator_id = conversation['initiatorWorkerId']

        # Ensure 'respondentQuestions', 'initiatorQuestions', and 'movieMentions' are dictionaries
        if isinstance(conversation.get('respondentQuestions'), dict) and isinstance(conversation.get('initiatorQuestions'), dict) and isinstance(conversation.get('movieMentions'), dict):
            # Track recommendations and movie mentions made by the respondent
            current_segment = []
            recommendation_count = 0
            seen_recommendations = set()
            seen_movie_mentions = set()
            seen_initiator_mentions = {}

            # Track the starting index for the next segment
            start_index = 0

            for msg_index, msg in enumerate(messages):
                current_segment.append(msg)

                # Track movies mentioned by the initiator
                if msg['senderWorkerId'] == initiator_id:
                    # Extract all movie mentions in the message text (e.g., "@123456")
                    mentioned_movies = re.findall(r"@(\d+)", msg['text'])
                    for movie_id in mentioned_movies:
                        if movie_id in conversation['movieMentions']:
                            seen_initiator_mentions[movie_id] = conversation['movieMentions'][movie_id]

                # Check if the current message contains a movie recommendation
                for movie_id, details in conversation['respondentQuestions'].items():
                    if (
                        details['suggested'] == 1
                        and f"@{movie_id}" in msg['text']
                        and movie_id not in seen_recommendations
                    ):
                        recommendation_count += 1

                        # Create a new conversation segment with messages from the current starting index to the current message
                        new_conversation = deepcopy(conversation)
                        new_conversation['messages'] = deepcopy(messages[start_index:msg_index + 1])
                        new_conversation['recommendationIndex'] = recommendation_count

                        # Include only the current recommendation in 'respondentQuestions'
                        new_conversation['respondentQuestions'] = {movie_id: details}

                        # Include only the current movie mention in 'movieMentions'
                        new_conversation['movieMentions'] = {
                            movie_id: conversation['movieMentions'][movie_id]
                        }

                        # Include the current movie in 'initiatorQuestions' if it exists
                        new_conversation['initiatorQuestions'] = {
                            movie_id: conversation['initiatorQuestions'][movie_id]
                        } if movie_id in conversation['initiatorQuestions'] else {}

                        # Add initiator movie mentions up to this point
                        new_conversation['initiatorMovieMentions'] = deepcopy(seen_initiator_mentions)

                        # Mark this recommendation and movie mention as processed
                        seen_recommendations.add(movie_id)
                        seen_movie_mentions.add(movie_id)

                        # Add the new conversation segment to the final list
                        new_data.append(new_conversation)

                        # Update the starting index for the next segment
                        start_index = msg_index + 1

                        # Break to move to the next recommendation point
                        break

    return new_data

In [ ]:
# Apply the function to the training data
data_train_split_no_overlap = split_conversations_no_overlap_with_initiator_mentions(data_train)

# Check the first few split conversations to confirm
for entry in data_train_split_no_overlap[:2]:  # Adjust indexing to see more
    print(entry)
    print("\n---\n")

{'movieMentions': {'165710': 'The Boss Baby (2017)'}, 'respondentQuestions': {'165710': {'suggested': 1, 'seen': 0, 'liked': 1}}, 'messages': [{'timeOffset': 0, 'text': "Hi there, how are you? I'm looking for movie recommendations", 'senderWorkerId': 0, 'messageId': 1021}, {'timeOffset': 15, 'text': 'I am doing okay. What kind of movies do you like?', 'senderWorkerId': 1, 'messageId': 1022}, {'timeOffset': 66, 'text': 'I like animations like @84779 and @191602', 'senderWorkerId': 0, 'messageId': 1023}, {'timeOffset': 86, 'text': 'I also enjoy @122159', 'senderWorkerId': 0, 'messageId': 1024}, {'timeOffset': 95, 'text': 'Anything artistic', 'senderWorkerId': 0, 'messageId': 1025}, {'timeOffset': 135, 'text': 'You might like @165710 that was a good movie.', 'senderWorkerId': 1, 'messageId': 1026}], 'conversationId': '391', 'respondentWorkerId': 1, 'initiatorWorkerId': 0, 'initiatorQuestions': {'165710': {'suggested': 1, 'seen': 0, 'liked': 1}}, 'recommendationIndex': 1, 'initiatorMovieMe

### Example Conversation 1
- **Movie Mentions**:
  - `165710`: "The Boss Baby (2017)"
- **Respondent Questions**:
  - `165710`: Suggested: 1, Seen: 0, Liked: 1
- **Initiator Movie Mentions**:
  - `84779`: "The Triplets of Belleville (2003)"
  - `191602`: "Waking Life (2001)"
  - `122159`: "Mary and Max (2009)"
- **Messages**:
  - **0s**: Initiator: "Hi there, how are you? I'm looking for movie recommendations"
  - **15s**: Respondent: "I am doing okay. What kind of movies do you like?"
  - **66s**: Initiator: "I like animations like @84779 and @191602"
  - **86s**: Initiator: "I also enjoy @122159"
  - **95s**: Initiator: "Anything artistic"
  - **135s**: Respondent: "You might like @165710 that was a good movie."
- **Conversation ID**: 391
- **Respondent Worker ID**: 1
- **Initiator Worker ID**: 0
- **Recommendation Index**: 1

---

# Neo4j Knowledge Graph

In [ ]:
# Neo4j Aura connection details
os.environ["OPENAI_API_KEY"] = pwd
uri = "neo4j+s://c87e3ce1.databases.neo4j.io"  # Replace with your URI
username = "neo4j"  # Default username for Neo4j
password = "zW13mCVhTWkcQLhIDoiKloD07IUg0W5TiszGzKwjCXA"  # Replace with your password

In [ ]:
!pip install py2neo

In [ ]:
from py2neo import Graph

# Connect to Neo4j
graph = Graph(uri, auth=(username, password))

# Updated query without LIMIT 5
query = """
MATCH (m:Movie)-[:`directed by`]->(d:Director)  // Fetch the director
WITH m, d

// Fetch top 3 related directors
OPTIONAL MATCH (d)-[r:COLLABORATES_WITH]->(rd:Director)
WITH m, d, COLLECT({RelatedDirector: rd.Director1})[0..3] AS TopRelatedDirectors

// Fetch top 3 movies that share genres (only using Genre_1)
OPTIONAL MATCH (m)-[:`belongs to`]->(g1:Genre_1)<-[:`belongs to`]-(sharedMovie:Movie)
WITH m, d, TopRelatedDirectors,
     COLLECT(DISTINCT {SharedMovie: sharedMovie.movie_name, Genre: g1.Genre1})[0..3] AS TopSharedMovies

// Fetch top 3 actors who acted in other movies
OPTIONAL MATCH (m)-[:`acted by`]->(a:Actor1)<-[:`acted by`]-(otherMovie:Movie)
WITH m, d, TopRelatedDirectors, TopSharedMovies,
     COLLECT({Actor: a.Star1, Movie: otherMovie.movie_name})[0..3] AS TopOtherActors

// Fetch actor-genre associations (only using Genre_1)
OPTIONAL MATCH (m)-[:`acted by`]->(a:Actor1)
OPTIONAL MATCH (m)-[:`belongs to`]->(g1:Genre_1)
WITH m, d, TopRelatedDirectors, TopSharedMovies, TopOtherActors,
     COLLECT(DISTINCT {Actor: a.Star1, Genre: g1.Genre1})[0..3] AS ActorGenres

// Fetch director-genre associations (only using Genre_1)
OPTIONAL MATCH (m)-[:`directed by`]->(d:Director)
OPTIONAL MATCH (m)-[:`belongs to`]->(g1:Genre_1)
WITH m, d, TopRelatedDirectors, TopSharedMovies, TopOtherActors, ActorGenres,
     COLLECT(DISTINCT g1.Genre1)[0..3] AS DirectorGenres

// Return the aggregated data without GenreAssociations
RETURN
  m.movie_id AS MovieID,
  m.movie_name AS Movie,
  d.Director1 AS Director,
  TopRelatedDirectors,
  TopSharedMovies,
  TopOtherActors,
  ActorGenres,
  DirectorGenres
"""

# Run the query
result = graph.run(query).data()

# Format the result as JSON
formatted_result = [
    {
        "MovieID": record["MovieID"],
        "Movie": record["Movie"],
        "Director": record["Director"],
        "TopRelatedDirectors": [
            director["RelatedDirector"] for director in record["TopRelatedDirectors"]
        ],
        "TopSharedMovies": record["TopSharedMovies"],
        "TopOtherActors": record["TopOtherActors"],
        "ActorGenres": record["ActorGenres"],
        "DirectorGenres": record["DirectorGenres"]
    }
    for record in result
]

# Print only the first 5 records from formatted_result
for i in formatted_result[:5]:  # Slicing the list to print only the first 5 entries
    print(i)


{'MovieID': 203371, 'Movie': 'Final Fantasy: The Spirits Within (2001)', 'Director': 'Hironobu Sakaguchi', 'TopRelatedDirectors': ['Tom McGrath', 'Tim Burton', 'Lawrence Guterman'], 'TopSharedMovies': [{'Genre': 'Animation', 'SharedMovie': 'The Triplets of Belleville (2003)'}, {'Genre': 'Animation', 'SharedMovie': 'Mary and Max (2009)'}, {'Genre': 'Animation', 'SharedMovie': 'Waking Life (2001)'}], 'TopOtherActors': [{'Actor': 'Alec Baldwin', 'Movie': 'The Boss Baby (2017)'}, {'Actor': 'Alec Baldwin', 'Movie': 'Beetlejuice (1988)'}, {'Actor': 'Alec Baldwin', 'Movie': 'Cats & Dogs (2001)'}], 'ActorGenres': [{'Actor': 'Alec Baldwin', 'Genre': 'Animation'}], 'DirectorGenres': ['Animation']}
{'MovieID': 84779, 'Movie': 'The Triplets of Belleville (2003)', 'Director': 'Sylvain Chomet', 'TopRelatedDirectors': [None], 'TopSharedMovies': [{'Genre': 'Animation', 'SharedMovie': 'Final Fantasy: The Spirits Within (2001)'}, {'Genre': 'Animation', 'SharedMovie': 'Mary and Max (2009)'}, {'Genre': 'A

we pass the conversation as part of our prompt in gpt and get a result

0) make json file with all movies that exist in our random sample of the test data, and add movie id for identification



1) first pull the movies mentioned from the conversation if liked != 0

2) extract relevant information for each movie mentioned through our json file



3) now pass relevant informaiton + conversation and other stuff into gpt for response

## Extract Test Data

In [ ]:
import random

# Set random seed for reproducibility
random.seed(77)

# Get a random sample of 98 entries
sampled_test_data = random.sample(data_test, 98)

In [ ]:
# Extracting movie IDs with liked value not equal to 0
liked_movie_mentions = {}

for entry in sampled_test_data:
    # Extract 'respondentQuestions' and 'initiatorQuestions'
    respondent_questions = entry.get('respondentQuestions', {})
    initiator_questions = entry.get('initiatorQuestions', {})

    # Ensure both are dictionaries before merging
    if isinstance(respondent_questions, dict) and isinstance(initiator_questions, dict):
        questions = {**respondent_questions, **initiator_questions}  # Merge the dictionaries
    else:
        questions = respondent_questions if isinstance(respondent_questions, dict) else {}
        questions.update(initiator_questions if isinstance(initiator_questions, dict) else {})

    # Filter movie IDs where liked value is not 0
    for movie_id, stats in questions.items():
        if stats.get('liked', 0) != 0:  # Check if 'liked' is not 0
            # Add to liked_movie_ids with mapping to movie name
            # Now liked_movie_ids contains the mapping of movieId to movie name for liked or ambivalent movies
            liked_movie_mentions[movie_id] = entry['movieMentions'].get(movie_id, "Unknown Movie")


using the keys of the mentioned movies from our sampled test data, we want to query our neo4j graph to create a json of relevant context information per movie mentioned.


In [ ]:
from py2neo import Graph
import json

# Connect to Neo4j
graph = Graph(uri, auth=(username, password))

# Initialize a list to store the results
all_results = []

# Loop through each movie ID in liked_movie_mentions.keys()
for movie_id in liked_movie_mentions.keys():
    movie_id = int(movie_id)
    # Updated query to filter by movie_id
    query = """
    MATCH (m:Movie)-[:`directed by`]->(d:Director)  // Fetch the director
    WHERE m.movie_id = $movie_id  // Use parameter placeholder
    WITH m, d

    // Fetch top 3 related directors
    OPTIONAL MATCH (d)-[r:COLLABORATES_WITH]->(rd:Director)
    WITH m, d, COLLECT({RelatedDirector: rd.Director1})[0..3] AS TopRelatedDirectors

    // Fetch top 3 movies that share genres (only using Genre_1)
    OPTIONAL MATCH (m)-[:`belongs to`]->(g1:Genre_1)<-[:`belongs to`]-(sharedMovie:Movie)
    WITH m, d, TopRelatedDirectors,
         COLLECT(DISTINCT {SharedMovie: sharedMovie.movie_name, Genre: g1.Genre1})[0..3] AS TopSharedMovies

    // Fetch top 3 actors who acted in other movies
    OPTIONAL MATCH (m)-[:`acted by`]->(a:Actor1)<-[:`acted by`]-(otherMovie:Movie)
    WITH m, d, TopRelatedDirectors, TopSharedMovies,
         COLLECT({Actor: a.Star1, Movie: otherMovie.movie_name})[0..3] AS TopOtherActors

    // Fetch actor-genre associations (only using Genre_1)
    OPTIONAL MATCH (m)-[:`acted by`]->(a:Actor1)
    OPTIONAL MATCH (m)-[:`belongs to`]->(g1:Genre_1)
    WITH m, d, TopRelatedDirectors, TopSharedMovies, TopOtherActors,
         COLLECT(DISTINCT {Actor: a.Star1, Genre: g1.Genre1})[0..3] AS ActorGenres

    // Fetch director-genre associations (only using Genre_1)
    OPTIONAL MATCH (m)-[:`directed by`]->(d:Director)
    OPTIONAL MATCH (m)-[:`belongs to`]->(g1:Genre_1)
    WITH m, d, TopRelatedDirectors, TopSharedMovies, TopOtherActors, ActorGenres,
         COLLECT(DISTINCT g1.Genre1)[0..3] AS DirectorGenres

    // Return the aggregated data without GenreAssociations
    RETURN
      m.movie_id AS MovieID,
      m.movie_name AS Movie,
      d.Director1 AS Director,
      TopRelatedDirectors,
      TopSharedMovies,
      TopOtherActors,
      ActorGenres,
      DirectorGenres
    """

    # Run the query with the specific movie_id
    result = graph.run(query, movie_id=movie_id).data()

    # Format the result as JSON for each movie
    formatted_result = [
        {
            "MovieID": record["MovieID"],
            "Movie": record["Movie"],
            "Director": record["Director"],
            "TopRelatedDirectors": [
                director["RelatedDirector"] for director in record["TopRelatedDirectors"]
            ],
            "TopSharedMovies": record["TopSharedMovies"],
            "TopOtherActors": record["TopOtherActors"],
            "ActorGenres": record["ActorGenres"],
            "DirectorGenres": record["DirectorGenres"]
        }
        for record in result
    ]

    # Append the formatted result for this movie to the all_results list
    all_results.extend(formatted_result)

# Write the filtered results to a JSON file
with open('/content/drive/MyDrive/DATASCI 266/266 Final Project/Code/filtered_query_results.json', 'w') as json_file:
    json.dump(all_results, json_file, indent=4)

# Optionally print the first 5 records
for i in all_results[:5]:  # Print only the first 5 entries from the result
    print(i)


{'MovieID': 142573, 'Movie': 'Love Actually (2003)', 'Director': 'Richard Curtis', 'TopRelatedDirectors': ['Mike Newell', 'Chris Columbus', 'Marc Lawrence'], 'TopSharedMovies': [{'Genre': 'Comedy', 'SharedMovie': 'Jerry Maguire (1996)'}, {'Genre': 'Comedy', 'SharedMovie': 'Caddyshack (1980)'}, {'Genre': 'Comedy', 'SharedMovie': 'Happy Gilmore (1996)'}], 'TopOtherActors': [{'Actor': 'Hugh Grant', 'Movie': 'Four Weddings and a Funeral (1994)'}, {'Actor': 'Hugh Grant', 'Movie': 'Music and Lyrics (2007)'}, {'Actor': 'Hugh Grant', 'Movie': 'Nine Months (1995)'}], 'ActorGenres': [{'Actor': 'Hugh Grant', 'Genre': 'Comedy'}], 'DirectorGenres': ['Comedy']}
{'MovieID': 205480, 'Movie': 'Titanic (2012)', 'Director': 'Kevin Lincoln', 'TopRelatedDirectors': [None], 'TopSharedMovies': [{'Genre': 'Adventure', 'SharedMovie': 'Girls Trip (2017)'}, {'Genre': 'Adventure', 'SharedMovie': '47 Meters Down (2017)'}, {'Genre': 'Adventure', 'SharedMovie': 'Far and Away (1992)'}], 'TopOtherActors': [{'Actor': N

In [ ]:
# Initialize the nested dictionary
conversation_liked_movies = {}

for entry in sampled_test_data:
    # Extract 'conversationId' and other fields
    conversation_id = entry.get('conversationId', "Unknown Conversation")
    respondent_questions = entry.get('respondentQuestions', {})
    initiator_questions = entry.get('initiatorQuestions', {})

    # Ensure both are dictionaries before merging
    if isinstance(respondent_questions, dict) and isinstance(initiator_questions, dict):
        questions = {**respondent_questions, **initiator_questions}  # Merge the dictionaries
    else:
        questions = respondent_questions if isinstance(respondent_questions, dict) else {}
        questions.update(initiator_questions if isinstance(initiator_questions, dict) else {})

    # Prepare liked_movie_mentions for this conversation
    liked_movie_mentions = {}

    for movie_id, stats in questions.items():
        if stats.get('liked', 0) != 0:  # Check if 'liked' is not 0
            # Map movie ID to its name
            liked_movie_mentions[movie_id] = entry['movieMentions'].get(movie_id, "Unknown Movie")

    # Add to the nested dictionary under the conversation ID
    conversation_liked_movies[conversation_id] = liked_movie_mentions

now from this, for each conversation that is passed into our LLM's prompt we only want to select the relevant context information

In [ ]:
def find_ground_truth(entry):
    respondent_questions = entry['respondentQuestions']

    # Skip if respondentQuestions is an empty list
    if isinstance(respondent_questions, list) and len(respondent_questions) == 0:
        print(f"Skipping conversation ID {entry['conversationId']} due to empty respondentQuestions.")
        return None

    # Process if respondentQuestions is a dictionary
    last_movie_id = None
    if isinstance(respondent_questions, dict):
        for movie_id, values in respondent_questions.items():
            if values['suggested'] == 1 and (values['liked'] == 1 or values['liked'] == 2):
                last_movie_id = str(movie_id)  # Update with the most recent match

    return last_movie_id  # Return the last match found, or None if no matches

In [ ]:
# Find Ground Truth (y_true)

y_true = []

# Loop through each entry in sampled_data and extract the ground truth with the conversation_id
for entry in sampled_test_data:
    conversation_id = entry['conversationId']  # Get the conversation ID
    ground_truth = find_ground_truth(entry)  # Get the ground truth movie ID

    # If we found a ground truth, add it along with the conversation_id
    if ground_truth:
        y_true.append({
            'conversation_id': conversation_id,
            'ground_truth': ground_truth
        })

# Print the final ground truth
print(f"Ground truth recommendations: {y_true}")

Ground truth recommendations: [{'conversation_id': '21498', 'ground_truth': '151989'}, {'conversation_id': '21787', 'ground_truth': '145997'}, {'conversation_id': '21239', 'ground_truth': '180527'}, {'conversation_id': '21417', 'ground_truth': '204870'}, {'conversation_id': '21222', 'ground_truth': '106233'}, {'conversation_id': '20943', 'ground_truth': '205163'}, {'conversation_id': '21680', 'ground_truth': '111041'}, {'conversation_id': '22361', 'ground_truth': '157454'}, {'conversation_id': '22815', 'ground_truth': '163184'}, {'conversation_id': '23059', 'ground_truth': '182858'}, {'conversation_id': '21412', 'ground_truth': '131811'}, {'conversation_id': '21043', 'ground_truth': '187422'}, {'conversation_id': '22801', 'ground_truth': '113701'}, {'conversation_id': '23186', 'ground_truth': '152202'}, {'conversation_id': '22482', 'ground_truth': '160690'}, {'conversation_id': '20051', 'ground_truth': '100821'}, {'conversation_id': '21647', 'ground_truth': '196900'}, {'conversation_id

In [ ]:
def get_kg_movie_context(movie_id, file_path):
    """
    Retrieve the contextual information for a specific movie ID from a JSON file.

    Parameters:
    - movie_id (int): The movie ID to search for.
    - file_path (str): Path to the JSON file containing movie data.

    Returns:
    - dict: The contextual information for the movie ID, or None if not found.
    """
    # Load the JSON file
    with open(file_path, 'r') as f:
        movie_data = json.load(f)

    # Search for the movie by its ID
    for movie in movie_data:
        if movie["MovieID"] == movie_id:
            return movie  # Return the full dictionary for the matched movie

    # Return None if the movie ID is not found
    return None


In [ ]:
# Example usage of get_movie_context
file_path = '/content/drive/MyDrive/DATASCI 266/266 Final Project/Code/filtered_query_results.json'
movie_id = 142573  # Replace with the desired movie ID

movie_context = get_kg_movie_context(movie_id, file_path)

if movie_context:
    print("Contextual Information:")
    for key, value in movie_context.items():
        print(f"{key}: {value}")
else:
    print(f"Movie ID {movie_id} not found.")

Contextual Information:
MovieID: 142573
Movie: Love Actually (2003)
Director: Richard Curtis
TopRelatedDirectors: ['Mike Newell', 'Chris Columbus', 'Marc Lawrence']
TopSharedMovies: [{'Genre': 'Comedy', 'SharedMovie': 'Jerry Maguire (1996)'}, {'Genre': 'Comedy', 'SharedMovie': 'Caddyshack (1980)'}, {'Genre': 'Comedy', 'SharedMovie': 'Happy Gilmore (1996)'}]
TopOtherActors: [{'Actor': 'Hugh Grant', 'Movie': 'Four Weddings and a Funeral (1994)'}, {'Actor': 'Hugh Grant', 'Movie': 'Music and Lyrics (2007)'}, {'Actor': 'Hugh Grant', 'Movie': 'Nine Months (1995)'}]
ActorGenres: [{'Actor': 'Hugh Grant', 'Genre': 'Comedy'}]
DirectorGenres: ['Comedy']


In [ ]:
def replace_ids_with_names_and_indicate_speaker(entry, liked_movie_mentions):
    # Initialize an empty array to store the context
    context = []

    # Get worker IDs
    initiator_id = entry["initiatorWorkerId"]
    respondent_id = entry["respondentWorkerId"]

    # Iterate through the messages in the entry
    for message in entry["messages"]:
        # Determine the role of the sender
        sender_id = message["senderWorkerId"]
        if sender_id == initiator_id:
            role = "initiator"
        elif sender_id == respondent_id:
            role = "recommender"
        else:
            role = "unknown"

        # Get the message text
        text = message["text"]

        # Replace any @<id> with the corresponding movie name
        for movie_id, movie_name in liked_movie_mentions.items():
            text = text.replace(f"@{movie_id}", movie_name)

        # Append the formatted string to the context array
        context.append(f"{role.capitalize()}: {text}")

    # Return the full context array
    return context

# Apply the function to the entry
context = replace_ids_with_names_and_indicate_speaker(entry, liked_movie_mentions)

# Print the context array to verify all lines
print(context)

['Recommender: Hello!', 'Initiator: Hello!', 'Recommender: What kind of movies do you like?', 'Initiator: I am looking for a movie recommendation.   When I was younger I really enjoyed the A Nightmare on Elm Street (1984)', 'Recommender: Oh, you like scary movies?', 'Recommender: I recently watched Happy Death Day  (2017)', 'Initiator: I also enjoyed watching The Last House on the Left  (1972)', 'Recommender: It was really good for a new &quot;scary movie&quot;', 'Initiator: I do enjoy some of the newer horror movies that I have seen as well.', 'Recommender: I heard that A Quiet Place (2018) is good. It is still in theaters though.', 'Initiator: I really liked the movie Annabelle  (2014)', 'Recommender: Me, too!', 'Recommender: It was really creepy, but I thought it was good!', 'Initiator: Or The Forest  (2016) I saw while in theaters, this was a very good movie.  It had me on the edge of my seat for the whole show.', "Recommender: I'm not sure if I saw that one, I'll have to check int

In [ ]:
movie_mentions = pd.read_csv('/content/drive/MyDrive/DATASCI 266/266 Final Project/redial_dataset/movies_with_mentions.csv')
movie_mentions = movie_mentions[['movieId', 'movieName']]
movie_mentions.head(5)

,movieId,movieName
0,75796,Headhunter (2009)
1,75815,Angels in the Outfield (1994)
2,75822,Eddie and the Cruisers (1983)
3,75828,Ninja Assassin (2009)
4,75867,Orgazmo (1997)


In [ ]:
'''
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

import os
os.kill(os.getpid(), 9)
'''

In [ ]:
import os
from openai import OpenAI

# Set up your OpenAI client
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"]  # Ensure your API key is correctly set
)

# Test the client with a simple request
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": "Hello, how are you?"}
    ]
)

print(response.choices[0].message.content.strip())

As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. Thanks for asking. How can I help you today?


# LLM Call

In [ ]:
def predict_movie(entry, conversation_liked_movies, file_path, movie_mentions):
    # Extract conversation ID first
    conversation_id = entry['conversationId']

    # Find the ground truth (true recommendation), find_ground_truth gives the movie_id
    true_recommendation = find_ground_truth({
        'respondentQuestions': entry['respondentQuestions'],
        'movieMentions': entry['movieMentions'],
        'messages': entry['messages'],
        'conversationId': conversation_id,
    })

    # Use conversation ID to pull the movies except for the true recommendation
    movies = {
        movie_id: details
        for movie_id, details in conversation_liked_movies[conversation_id].items()
        if movie_id != true_recommendation
    }

    # Use the subset of movies to create the context without the true recommendation
    conversation = replace_ids_with_names_and_indicate_speaker(entry, movies)

    # Now pull the KG context for each movie in movies
    merged_kg_context = []
    for movie_id in movies.keys():
        merged_kg_context.append(get_kg_movie_context(movie_id, file_path))

    # Format the prompt to include all movies from the full movie list
    movie_id_name_mapping = "\n".join([f"{movie_id}, {movie_name}" for movie_id, movie_name in movie_mentions.items()])

    ##### LLM Zero-shot Recommendation ####
    prompt = f"""
    Pretend you are a movie recommender system.
    I will give you a conversation between a user and you (a recommender system).
    Based on the conversation, you reply me with 5 recommendations without extra sentences.
    The recommendations must follow this format:
    ---
    1. Movie 1 ID, Movie 1 Name
    2. Movie 2 ID, Movie 2 Name
    3. Movie 3 ID, Movie 3 Name
    4. Movie 4 ID, Movie 4 Name
    5. Movie 5 ID, Movie 5 Name
    ---
    Here is the conversation:

    {conversation}

    Here is additional information to assist your recommendation:

    {merged_kg_context}

    All movies you can recommend (movie id and name):

    {movie_id_name_mapping}

    Please provide 5 recommendations:
    """

    response = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt
        }],
        model="gpt-4o"  # Specify the model you want to use
    )

    # Extract and return the recommendation
    return response.choices[0].message.content.strip()


In [ ]:
# Example usage
file_path = '/content/drive/MyDrive/DATASCI 266/266 Final Project/Code/filtered_query_results.json'
predict_movie(sampled_test_data[0], conversation_liked_movies, file_path, movie_mentions)

'```\n1. 75815, Angels in the Outfield  (1994)\n2. 74804, Pride & Prejudice  (2005)\n3. 68734, A Walk to Remember  (2002)\n4. 69340, Dear John  (2010)\n5. 74258, The Vow  (2012)\n```'

In [ ]:
# Getting all predictions

# Initialize an empty dictionary to store the predictions
predictions = {}

# Loop through each entry in the sampled_test_data
for entry in sampled_test_data:
    # Get the conversation ID for the entry
    conversation_id = entry['conversationId']

    # Call the predict_movie function and store the result
    prediction = predict_movie(entry, conversation_liked_movies, file_path, movie_mentions)

    # Store the prediction in the dictionary
    predictions[conversation_id] = prediction

    # Print the conversation ID and the corresponding prediction
    print(f"Conversation ID: {conversation_id}")
    print(f"Prediction:\n{prediction}")
    print("\n" + "="*50 + "\n")

# At the end, predictions will hold all conversation IDs mapped to their respective predictions.

Conversation ID: 21498
Prediction:
---
1. 75815, Angels in the Outfield (1994)
2. 75822, Eddie and the Cruisers (1983)
3. 75828, Ninja Assassin (2009)
4. 75867, Orgazmo (1997)
5. 75896, Cabin Fever 2: Spring Fever (2009)
---


Conversation ID: 21787
Prediction:
---
1. 969, Singin' in the Rain (1952)
2. 1652, My Fair Lady (1964)
3. 1784, The Sound of Music (1965)
4. 6197, The King and I (1956)
5. 4935, Chicago (2002)
---


Conversation ID: 21239
Prediction:
---
1. 75815, Angels in the Outfield (1994)
2. 75822, Eddie and the Cruisers (1983)
3. 75828, Ninja Assassin (2009)
4. 75867, Orgazmo (1997)
5. 206079, 2036 Origin Unknown (2018)
---


Conversation ID: 21417
Prediction:
---
1. 75828, Ninja Assassin (2009)
2. 75867, Orgazmo (1997)
3. 206079, 2036 Origin Unknown (2018)
4. 206085, Hotel Artemis (2018)
5. 206087, Hereditary (2018)
---


Conversation ID: 21222
Prediction:
---
1. 75815, Angels in the Outfield (1994)
2. 203844, Maleficent (2014)
3. 71679, The Chronicles of Narnia: The Lion,

In [ ]:
import re

# Initialize an evaluation list to store results
evaluation = []

# Loop through each ground truth entry in y_true
for item in y_true:
    conversation_id = item['conversation_id']
    ground_truth = item['ground_truth']

    # Get the predictions for the corresponding conversation ID
    if conversation_id in predictions:
        prediction = predictions[conversation_id]

        # Use regex to extract movie IDs from the prediction
        predicted_ids = re.findall(r'\b\d+\b', prediction)

        # Check if the ground truth is present in the predicted IDs
        is_present = ground_truth in predicted_ids

        # Append the result to the evaluation list
        evaluation.append({
            "conversation_id": conversation_id,
            "ground_truth": ground_truth,
            "predicted_ids": predicted_ids,
            "is_present": is_present
        })

# Print the evaluation results
for result in evaluation:
    print(f"Conversation ID: {result['conversation_id']}")
    print(f"Ground Truth: {result['ground_truth']}")
    print(f"Predicted IDs: {result['predicted_ids']}")
    print(f"Is Ground Truth Present: {result['is_present']}")
    print("\n" + "="*50 + "\n")

Conversation ID: 21498
Ground Truth: 151989
Predicted IDs: ['1', '75815', '1994', '2', '75822', '1983', '3', '75828', '2009', '4', '75867', '1997', '5', '75896', '2', '2009']
Is Ground Truth Present: False


Conversation ID: 21787
Ground Truth: 145997
Predicted IDs: ['1', '969', '1952', '2', '1652', '1964', '3', '1784', '1965', '4', '6197', '1956', '5', '4935', '2002']
Is Ground Truth Present: False


Conversation ID: 21239
Ground Truth: 180527
Predicted IDs: ['1', '75815', '1994', '2', '75822', '1983', '3', '75828', '2009', '4', '75867', '1997', '5', '206079', '2036', '2018']
Is Ground Truth Present: False


Conversation ID: 21417
Ground Truth: 204870
Predicted IDs: ['1', '75828', '2009', '2', '75867', '1997', '3', '206079', '2036', '2018', '4', '206085', '2018', '5', '206087', '2018']
Is Ground Truth Present: False


Conversation ID: 21222
Ground Truth: 106233
Predicted IDs: ['1', '75815', '1994', '2', '203844', '2014', '3', '71679', '2005', '4', '79132', '2007', '5', '180874', '2016

In [ ]:
# Filter only the "true" cases where the ground truth is present in the predictions
true_predictions = [result for result in evaluation if result['is_present']]

# Print the true predictions
for result in true_predictions:
    print(f"Conversation ID: {result['conversation_id']}")
    print(f"Ground Truth: {result['ground_truth']}")
    print(f"Predicted IDs: {result['predicted_ids']}")
    print(f"Is Ground Truth Present: {result['is_present']}")
    print("\n" + "="*50 + "\n")

Conversation ID: 20820
Ground Truth: 159885
Predicted IDs: ['1', '75815', '3', '2010', '2', '75867', '2001', '3', '159885', '2009', '4', '192261', '2016', '5', '206085', '2017']
Is Ground Truth Present: True


Conversation ID: 22295
Ground Truth: 130591
Predicted IDs: ['1', '130591', '1984', '2', '205075', '1980', '3', '204928', '1982', '4', '204547', '1976', '5', '204629', '1978']
Is Ground Truth Present: True


Conversation ID: 22619
Ground Truth: 160690
Predicted IDs: ['1', '204870', '2009', '2', '160690', '2007', '3', '75815', '2008', '4', '75867', '40', '2005', '5', '75828', '2007']
Is Ground Truth Present: True


Conversation ID: 21478
Ground Truth: 204870
Predicted IDs: ['1', '75828', '2009', '2', '75867', '1997', '3', '206085', '2018', '4', '204870', '2016', '5', '206087', '2018']
Is Ground Truth Present: True


Conversation ID: 20194
Ground Truth: 181301
Predicted IDs: ['1', '181301', '2007', '2', '75828', '2009', '3', '206087', '2018', '4', '75867', '1997', '5', '75822', '198

In [ ]:
# Calculate the percentage of true predictions
total_predictions = len(evaluation)
correct_predictions = len(true_predictions)
accuracy_percentage = (correct_predictions / total_predictions) * 100

print(f"Total Predictions: {total_predictions}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Recall@5: {accuracy_percentage:.2f}%")

Total Predictions: 98
Correct Predictions: 8
Recall@5: 8.16%


**Total Predictions: 98**

**Correct Predictions: 8**

**Recall at 5: 8.16%**

## Baseline Model Predictions

In [ ]:
def predict_movie_without_context(entry, conversation_liked_movies, movie_mentions):
    # Extract conversation ID first
    conversation_id = entry['conversationId']

    # Find the ground truth (true recommendation), find_ground_truth gives the movie_id
    true_recommendation = find_ground_truth({
        'respondentQuestions': entry['respondentQuestions'],
        'movieMentions': entry['movieMentions'],
        'messages': entry['messages'],
        'conversationId': conversation_id,
    })

    # Use conversation ID to pull the movies except for the true recommendation
    movies = {
        movie_id: details
        for movie_id, details in conversation_liked_movies[conversation_id].items()
        if movie_id != true_recommendation
    }

    # Use the subset of movies to create the context without the true recommendation
    conversation = replace_ids_with_names_and_indicate_speaker(entry, movies)

    # Format the prompt to include all movies from the full movie list
    movie_id_name_mapping = "\n".join([f"{movie_id}, {movie_name}" for movie_id, movie_name in movie_mentions.items()])

    ##### LLM Zero-shot Recommendation ####
    prompt = f"""
    Pretend you are a movie recommender system.
    I will give you a conversation between a user and you (a recommender system).
    Based on the conversation, you reply me with 5 recommendations without extra sentences.
    The recommendations must follow this format:
    ---
    1. Movie 1 ID, Movie 1 Name
    2. Movie 2 ID, Movie 2 Name
    3. Movie 3 ID, Movie 3 Name
    4. Movie 4 ID, Movie 4 Name
    5. Movie 5 ID, Movie 5 Name
    ---
    Here is the conversation:

    {conversation}

    All movies you can recommend (movie id and name):

    {movie_id_name_mapping}

    Please provide 5 recommendations:
    """

    response = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": prompt
        }],
        model="gpt-4o"  # Specify the model you want to use
    )

    # Extract and return the recommendation
    return response.choices[0].message.content.strip()

In [ ]:
# Getting all predictions without context

# Initialize an empty dictionary to store the predictions
predictions_without_context = {}

# Loop through each entry in the sampled_test_data
for entry in sampled_test_data:
    # Get the conversation ID for the entry
    conversation_id = entry['conversationId']

    # Call the predict_movie_without_context function and store the result
    prediction = predict_movie_without_context(entry, conversation_liked_movies, movie_mentions)

    # Store the prediction in the dictionary
    predictions_without_context[conversation_id] = prediction

    # Print the conversation ID and the corresponding prediction
    print(f"Conversation ID: {conversation_id}")
    print(f"Prediction:\n{prediction}")
    print("\n" + "="*50 + "\n")

# At the end, predictions_without_context will hold all conversation IDs mapped to their respective predictions.

Conversation ID: 21498
Prediction:
---
1. 75815, Angels in the Outfield (1994)
2. 75822, Eddie and the Cruisers (1983)
3. 75828, Ninja Assassin (2009)
4. 75867, Orgazmo (1997)
5. 206092, Mowgli (2019)
---


Conversation ID: 21787
Prediction:
```
1. 75828, Eddie and the Cruisers (1983)
2. 206080, The Hatching (2016)
3. 75815, Angels in the Outfield (1994)
4. 75822, Ninja Assassin (2009)
5. 206092, Mowgli (2019)
```


Conversation ID: 21239
Prediction:
```
1. 2, Eddie and the Cruisers (1983)
2. 3, Ninja Assassin (2009)
3. 4, Orgazmo (1997)
4. 6919, 2036 Origin Unknown (2018)
5. 6922, Hereditary (2018)
```


Conversation ID: 21417
Prediction:
```
1. 75828, Ninja Assassin (2009)
2. 206087, Hereditary (2018)
3. 75815, Angels in the Outfield (1994)
4. 75867, Orgazmo (1997)
5. 206079, 2036 Origin Unknown (2018)
```


Conversation ID: 21222
Prediction:
---
1. 75867, The Hatching (2016)
2. 6919, 2036 Origin Unknown (2018)
3. 6918, The 5th Wave (2016)
4. 13345, Warcraft: The Beginning (2016)
5. 

In [ ]:
import re

def evaluate_predictions(predictions, y_true):
    # Initialize an evaluation list to store results
    evaluation = []

    # Loop through each ground truth entry in y_true
    for item in y_true:
        conversation_id = item['conversation_id']
        ground_truth = item['ground_truth']

        # Get the predictions for the corresponding conversation ID
        if conversation_id in predictions:
            prediction = predictions[conversation_id]

            # Use regex to extract movie IDs from the prediction
            predicted_ids = re.findall(r'\b\d+\b', prediction)

            # Check if the ground truth is present in the predicted IDs
            is_present = ground_truth in predicted_ids

            # Append the result to the evaluation list
            evaluation.append({
                "conversation_id": conversation_id,
                "ground_truth": ground_truth,
                "predicted_ids": predicted_ids,
                "is_present": is_present
            })

    # Filter only the "true" cases where the ground truth is present in the predictions
    true_predictions = [result for result in evaluation if result['is_present']]

    # Calculate the percentage of true predictions
    total_predictions = len(evaluation)
    correct_predictions = len(true_predictions)
    accuracy_percentage = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0

    # Return detailed evaluation and summary statistics
    return {
        "evaluation": evaluation,
        "true_predictions": true_predictions,
        "accuracy_percentage": accuracy_percentage
    }

In [ ]:
# Evaluate predictions with context
results_with_context = evaluate_predictions(predictions, y_true)
print(f"Recall@5 with context: {results_with_context['accuracy_percentage']:.2f}%")

# Evaluate predictions without context
results_without_context = evaluate_predictions(predictions_without_context, y_true)
print(f"Recall@5 without context: {results_without_context['accuracy_percentage']:.2f}%")

Recall@5 with context: 8.16%
Recall@5 without context: 3.06%


## Measuring Hallucination

### ID Hallucination

In [ ]:
def extract_predicted_ids(predictions):
    # Initialize a list to store all predicted IDs
    all_predicted_ids = []

    # Loop through each prediction in the dictionary
    for conversation_id, prediction in predictions.items():
        # Use regex to extract movie IDs from the prediction
        predicted_ids = re.findall(r'\b\d+\b', prediction)
        all_predicted_ids.extend(predicted_ids)

    return all_predicted_ids

# Extract all predicted IDs from predictions
predicted_ids_with_context = extract_predicted_ids(predictions)

# Extract all predicted IDs from predictions_without_context
predicted_ids_without_context = extract_predicted_ids(predictions_without_context)

# Print results
print(f"Predicted IDs with context: {predicted_ids_with_context}")
print(f"Predicted IDs without context: {predicted_ids_without_context}")

Predicted IDs with context: ['1', '75815', '1994', '2', '75822', '1983', '3', '75828', '2009', '4', '75867', '1997', '5', '75896', '2', '2009', '1', '969', '1952', '2', '1652', '1964', '3', '1784', '1965', '4', '6197', '1956', '5', '4935', '2002', '1', '75815', '1994', '2', '75822', '1983', '3', '75828', '2009', '4', '75867', '1997', '5', '206079', '2036', '2018', '1', '75828', '2009', '2', '75867', '1997', '3', '206079', '2036', '2018', '4', '206085', '2018', '5', '206087', '2018', '1', '75815', '1994', '2', '203844', '2014', '3', '71679', '2005', '4', '79132', '2007', '5', '180874', '2016', '1', '75822', '1983', '2', '75815', '1994', '3', '75828', '2009', '4', '206087', '2018', '5', '75796', '2009', '1', '114814', '2001', '2', '59103', '2002', '3', '27530', '1998', '4', '89745', '2007', '5', '8961', '2001', '1', '75867', '2006', '2', '6922', '2018', '3', '75896', '1982', '4', '75815', '2015', '5', '206079', '2019', '1', '95120', '2', '2017', '2', '152081', '2018', '3', '106100', '201

In [ ]:
# Function to filter out IDs of length one and years
def filter_invalid_ids(predicted_ids):
    return [
        movie_id for movie_id in predicted_ids
        if len(movie_id) > 1 and not (len(movie_id) == 4 and (movie_id.startswith("19") or movie_id.startswith("20")))
    ]

# Apply the filtering function to both lists
filtered_ids_with_context = filter_invalid_ids(predicted_ids_with_context)
filtered_ids_without_context = filter_invalid_ids(predicted_ids_without_context)

# Print the filtered results
print(f"Filtered IDs with context: {filtered_ids_with_context}")
print(f"Filtered IDs without context: {filtered_ids_without_context}")

Filtered IDs with context: ['75815', '75822', '75828', '75867', '75896', '969', '1652', '1784', '6197', '4935', '75815', '75822', '75828', '75867', '206079', '75828', '75867', '206079', '206085', '206087', '75815', '203844', '71679', '79132', '180874', '75822', '75815', '75828', '206087', '75796', '114814', '59103', '27530', '89745', '8961', '75867', '6922', '75896', '75815', '206079', '95120', '152081', '106100', '129396', '58668', '694', '692', '689', '690', '682', '1408', '75929', '84748', '133248', '160554', '179270', '75815', '75822', '75828', '206079', '206087', '75867', '75828', '75815', '206080', '206079', '75815', '75822', '75828', '75867', '206087', '6919', '75796', '75815', '75822', '75828', '75867', '206087', '75828', '75815', '206080', '75822', '75867', '75828', '206085', '206087', '75896', '171495', '139639', '146316', '163109', '165933', '75828', '75815', '206092', '75822', '206087', '75796', '75815', '75822', '75828', '75867', '75822', '75867', '75896', '75909', '75979'

In [ ]:
len(filtered_ids_with_context), len(filtered_ids_without_context)

(490, 489)

In [ ]:
all_ids = movie_mentions.movieId.to_list()
all_ids = [str(movie_id) for movie_id in all_ids]
all_ids

['75796',
 '75815',
 '75822',
 '75828',
 '75867',
 '75881',
 '75903',
 '75918',
 '75962',
 '75968',
 '75981',
 '75984',
 '76012',
 '76019',
 '76023',
 '76036',
 '76042',
 '76067',
 '76069',
 '76071',
 '76102',
 '76111',
 '76113',
 '76130',
 '76137',
 '76143',
 '76145',
 '76160',
 '76170',
 '76171',
 '76177',
 '76187',
 '76195',
 '76198',
 '76279',
 '76284',
 '76325',
 '76336',
 '76339',
 '76356',
 '76365',
 '76391',
 '76481',
 '76483',
 '76488',
 '76508',
 '76512',
 '76519',
 '76524',
 '76530',
 '76620',
 '76627',
 '76636',
 '76639',
 '76658',
 '76660',
 '76706',
 '76724',
 '76746',
 '76757',
 '76772',
 '76781',
 '76802',
 '76825',
 '76829',
 '76845',
 '76847',
 '76849',
 '76861',
 '76887',
 '76935',
 '76940',
 '77043',
 '77045',
 '77066',
 '77115',
 '77128',
 '77136',
 '77140',
 '77141',
 '77154',
 '77161',
 '77176',
 '77190',
 '77195',
 '77216',
 '77253',
 '77267',
 '77293',
 '77306',
 '77330',
 '77352',
 '77358',
 '77363',
 '77376',
 '77384',
 '77388',
 '77415',
 '77455',
 '77478',


In [ ]:
# Function to find invalid IDs not present in all_ids
def find_invalid_ids(filtered_ids, all_ids):
    return [movie_id for movie_id in filtered_ids if movie_id not in all_ids]

# Identify invalid IDs for both filtered lists
invalid_ids_with_context = find_invalid_ids(filtered_ids_with_context, all_ids)
invalid_ids_without_context = find_invalid_ids(filtered_ids_without_context, all_ids)

# Print the results
print(f"Invalid IDs with context: {invalid_ids_with_context}")
print(f"Invalid IDs without context: {invalid_ids_without_context}")

Invalid IDs with context: ['75896', '969', '1652', '1784', '6197', '4935', '203844', '71679', '79132', '180874', '114814', '59103', '27530', '89745', '8961', '6922', '75896', '95120', '152081', '106100', '129396', '58668', '694', '692', '689', '690', '682', '1408', '75929', '84748', '133248', '160554', '179270', '6919', '75896', '171495', '139639', '146316', '163109', '165933', '75896', '75909', '75979', '76054', '75798', '75896', '6213', '4378', '5516', '120979', '119579', '129028', '103659', '141998', '76136', '40', '142983', '76077', '76500', '76526', '76532', '76548', '76561', '75991', '75819', '84152', '75985', '75820', '76093', '76200', '78039', '192261', '1862', '99007', '145356', '140607', '93363', '90015', '161044', '148626', '121766', '112191', '74818', '100304', '975265', '75891', '75829', '75988', '76054', '205997', '204945', '205746', '205689', '206011', '205075', '204928', '204547', '204629', '205144', '203884', '75896', '6922', '6919', '40', '75896', '75499', '72408', '7

In [ ]:
# Calculate the degree of hallucination
degree_of_hallucination_with_context = len(invalid_ids_with_context) / len(filtered_ids_with_context) * 100
degree_of_hallucination_without_context = len(invalid_ids_without_context) / len(filtered_ids_without_context) * 100

# Print the results
# print(f"Invalid IDs with context: {invalid_ids_with_context}")
print(f"Degree of hallucination with context: {degree_of_hallucination_with_context:.2f}%")

# print(f"Invalid IDs without context: {invalid_ids_without_context}")
print(f"Degree of hallucination without context: {degree_of_hallucination_without_context:.2f}%")

Degree of hallucination with context: 31.22%
Degree of hallucination without context: 16.16%


### Movie Name Hallucination

In [ ]:
movie_mentions

,movieId,movieName
0,75796,Headhunter (2009)
1,75815,Angels in the Outfield (1994)
2,75822,Eddie and the Cruisers (1983)
3,75828,Ninja Assassin (2009)
4,75867,Orgazmo (1997)
...,...,...
6919,206079,2036 Origin Unknown (2018)
6920,206080,The Hatching (2016)
6921,206085,Hotel Artemis (2018)
6922,206087,Hereditary (2018)


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=movie_mentions)

https://docs.google.com/spreadsheets/d/16QAmzD6f6Jj6H2Tr1pC_CyiUH7m7UvlP7k_-epRhD7w#gid=0


In [ ]:
# Function to normalize movie names and remove years in parentheses
def normalize_name(name):
    # Remove years in parentheses, strip extra spaces, and convert to lowercase
    return re.sub(r'\s*\(\d{4}\)', '', name).strip().lower()

# Normalize valid movie names
normalized_valid_movie_names = [normalize_name(name) for name in movie_mentions['movieName'].tolist()]

# Extract and normalize movie names from predictions
def extract_and_normalize_movie_names(prediction):
    # Extract the movie names from the format: "ID, Movie Name"
    movie_names = re.findall(r'\d+,\s*(.*)', prediction)
    return [normalize_name(name) for name in movie_names]

# Process predictions with context
normalized_predicted_names_with_context = []
for prediction in predictions.values():
    normalized_predicted_names_with_context.extend(extract_and_normalize_movie_names(prediction))

# Process predictions without context
normalized_predicted_names_without_context = []
for prediction in predictions_without_context.values():
    normalized_predicted_names_without_context.extend(extract_and_normalize_movie_names(prediction))

# Identify hallucinated names
hallucinated_names_with_context = [name for name in normalized_predicted_names_with_context if name not in normalized_valid_movie_names]
hallucinated_names_without_context = [name for name in normalized_predicted_names_without_context if name not in normalized_valid_movie_names]

# Calculate the degree of hallucination
degree_of_hallucination_names_with_context = len(hallucinated_names_with_context) / len(normalized_predicted_names_with_context) * 100
degree_of_hallucination_names_without_context = len(hallucinated_names_without_context) / len(normalized_predicted_names_without_context) * 100

# Print results
print(f"Hallucinated names with context: {hallucinated_names_with_context}")
print(f"Degree of hallucination (names) with context: {degree_of_hallucination_names_with_context:.2f}%")

print(f"Hallucinated names without context: {hallucinated_names_without_context}")
print(f"Degree of hallucination (names) without context: {degree_of_hallucination_names_without_context:.2f}%")

Hallucinated names with context: ['cabin fever 2: spring fever', 'summer lovers', 'midsommar', 'mary poppins returns', 'beatriz at dinner', 'robot & frank', 'the incredibles 2', 'ad astra', 'heathers the musical', 'another horror movie not listed', 'crazy, stupid, love.', 'once upon a time in hollywood', 'in the mouth of madness', "ang lee's life of pi"]
Degree of hallucination (names) with context: 2.86%
Hallucinated names without context: ['warcraft: the beginning', 'the perfect date', "a dog's journey", 'mickey and the beanstalk', 'se7en', 'unknown thriller', 'monsters vs aliens', 'crazy, stupid, love.']
Degree of hallucination (names) without context: 1.63%


# Quality Assurance

In [ ]:
for i in movies.keys():
  print(i)

142573
202418
205480
175096


In [ ]:
sampled_test_data[0]

{'movieMentions': {'142573': 'Love Actually (2003)',
  '202418': 'Notebook  (2006)',
  '205480': 'Titanic (2012)',
  '175096': 'Titanic  (1997)',
  '151989': 'Dear John  (2010)'},
 'respondentQuestions': {'142573': {'suggested': 1, 'seen': 1, 'liked': 1},
  '202418': {'suggested': 0, 'seen': 1, 'liked': 1},
  '205480': {'suggested': 1, 'seen': 2, 'liked': 2},
  '175096': {'suggested': 0, 'seen': 1, 'liked': 1},
  '151989': {'suggested': 1, 'seen': 1, 'liked': 1}},
 'messages': [{'timeOffset': 0,
   'text': 'Hi',
   'senderWorkerId': 1024,
   'messageId': 214289},
  {'timeOffset': 1,
   'text': 'Hello',
   'senderWorkerId': 972,
   'messageId': 214290},
  {'timeOffset': 10,
   'text': 'How are you doing tonight?',
   'senderWorkerId': 972,
   'messageId': 214291},
  {'timeOffset': 28,
   'text': 'I am looking for good movie to watch',
   'senderWorkerId': 1024,
   'messageId': 214292},
  {'timeOffset': 41,
   'text': 'Something like @202418',
   'senderWorkerId': 1024,
   'messageId': 2

In [ ]:
entry = sampled_test_data[0]
true_recommendation = find_ground_truth({
      'respondentQuestions': entry['respondentQuestions'],
      'movieMentions': entry['movieMentions'],
      'messages': entry['messages'],
      'conversationId': conversation_id,
  })
print(true_recommendation)

<class 'str'>


In [ ]:
true_recommendation

'151989'

In [ ]:
# Assuming conversation_liked_movies and conversation_id are defined
# and true_recommendation contains the movie ID to exclude.

# Extract movies for the given conversation_id
movies = {
    movie_id: details
    for movie_id, details in conversation_liked_movies[conversation_id].items()
    if movie_id != true_recommendation
}

# Now `movies` will exclude the true_recommendation.
movies

{'142573': 'Love Actually (2003)',
 '202418': 'Notebook  (2006)',
 '205480': 'Titanic (2012)',
 '175096': 'Titanic  (1997)'}

In [ ]:
conversation_liked_movies[conversation_id]

{'142573': 'Love Actually (2003)',
 '202418': 'Notebook  (2006)',
 '205480': 'Titanic (2012)',
 '175096': 'Titanic  (1997)',
 '151989': 'Dear John  (2010)'}

In [ ]:
conversation_id = entry['conversationId']
print(conversation_id)
movies = conversation_liked_movies[conversation_id] # dictionary of movie ids as keys and movie names as values
print(movies)
conversation = replace_ids_with_names_and_indicate_speaker(entry, movies)
print(conversation)

21498
{'142573': 'Love Actually (2003)', '202418': 'Notebook  (2006)', '205480': 'Titanic (2012)', '175096': 'Titanic  (1997)', '151989': 'Dear John  (2010)'}
['Initiator: Hi', 'Recommender: Hello', 'Recommender: How are you doing tonight?', 'Initiator: I am looking for good movie to watch', 'Initiator: Something like Notebook  (2006)', 'Recommender: oh', 'Recommender: i liked that one', 'Recommender: Have you seen Dear John  (2010)', 'Initiator: Yes, I love that movie', 'Recommender: me too :D my favorite', 'Recommender: Have you heard of Love Actually (2003) or the classic Titanic (2012)', 'Initiator: No, I have not, I will watch it', 'Initiator: You can watch Titanic  (1997) if you have not already.', 'Recommender: I have i love it', 'Recommender: !', 'Recommender: thats it see ya!']
